In [ ]:
from pyspark import SparkContext
sc = SparkContext()

### NumPy (Numerical Python) - Basics

In [ ]:
import numpy
from numpy import array

In [ ]:
list_of_list = [[1,2], [4,9], [10,100]]  # list of list
# list_of_list + 1
# list_of_list * 2

# A numpy array is similar to MATLAB matrix
numpy_array = array(list_of_list)
print(numpy_array)
print('Add 1 to each element')
print(numpy_array + 1)
print('-------')
print('Multiply each element by 2')
print(numpy_array * 2)

In [ ]:
print('Shape of array is ', numpy_array.shape)
print('Rank of array (matrix) is ', numpy_array.ndim)
print('Item size is ', numpy_array.dtype)
print('Reshaped Array from 3x2 into 2x3:')
print(numpy_array.reshape(2,3))

#### numpy array operations

In [ ]:
A = array( [[1,1],
               [0,1]] )
B = array( [[2,0],
                   [3,4]] )

print('A')
print(A)
print('B')
print(B)
print('\n')
print('A*B')
print(A*B)
print('\n')
print('A-B')
print(A-B)
print('\n')
print('A.B')
print(A.dot(B))

In [ ]:
# Some more statistical functions
print('Sum of matrix A is ', A.sum())
print('Sum of matrix B is ', B.sum())
print('Min element in marix B is ', B.min())
print('Max element in marix B is ', B.max())
print('Exponential for B is ', numpy.exp(B))
print('Exponential for B is ', numpy.sqrt(B))

## 1. Logistic Regression

https://machinelearningmastery.com/logistic-regression-for-machine-learning/

Spark MLLIB Docs: https://spark.apache.org/docs/2.2.0/ml-classification-regression.html#logistic-regression

#### Load the training and test data set in raw form

In [ ]:
raw_data = sc.textFile('data/kddcup10.data.gz')
test_raw_data = sc.textFile('data/kddtest.gz')

In [ ]:
raw_data.take(10)  # Check how the data looks like

In [ ]:
raw_data.filter(lambda x: 'normal' not in x).take(10)  # Check how the 'not' normal data looks like

In [ ]:
test_raw_data.take(5)

#### Prepare the raw training and test data for usage with the classifier

In [ ]:
from pyspark.mllib.regression import LabeledPoint

# Write down the code for parsing here, "ONLY" the numerical columns have to be selected for training purposes!
def parse_interaction(line):
    """ Selects the numerical columns and converts the last column (attack type) to either 0.0 or 1.0 (output label)
    If the attack type is normal. then label should be 0.0 for every other case it should be 1.0
    
    Args: line of the data set in a string format
    
    Returns: LabeledPoint object where the first argument is the output label (Double) and 
    second argument is an numpy array of remaining feature columns (Double)
    """
     # leave_out columns 1,2,3,41
    line_split = line.split(",")
   
    clean_line_split = line_split[0:1]+line_split[4:41]
    output_attack = 1.0
    if line_split[41]=='normal.':
        output_attack = 0.0
    
    features_numpy_array = array([float(x) for x in clean_line_split])   
    return LabeledPoint(output_attack, features_numpy_array)

training_data = raw_data.map(parse_interaction)
test_data = test_raw_data.map(parse_interaction)

#### Train the Logistic Regression classfier with the training dataset

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
logit_model = LogisticRegressionWithLBFGS.train(training_data)
logit_model

#### Predict the output labels for test datasets with the logistic regression classifier model object

In [ ]:
# logit_model.predict will predict the output label based on the features given to it
original_labels_vs_predicted_labels = test_data.map(lambda p: (p.label, logit_model.predict(p.features)))
original_labels_vs_predicted_labels.take(5)

#### Test the accuracy by selecting only the records where the given labels and predicted labels are equal , and dividing the count of those records by total records

In [ ]:
test_accuracy = original_labels_vs_predicted_labels.filter(lambda x: x[0] == x[1]).count() / float(test_data.count())

In [ ]:
print(round(test_accuracy,4))  # Print the accuracy

## k-means Clustering with Mlib

https://en.wikipedia.org/wiki/K-means_clustering

K-means is one of the most commonly used clustering algorithms that clusters the data points into a predefined number of clusters. 

Spark MLLIB Docs: https://spark.apache.org/docs/latest/mllib-clustering.html#k-means

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pyspark.mllib.clustering import KMeans, KMeansModel
from math import sqrt

In [ ]:
rdd = sc.textFile('data/unbalance.txt')
rdd.take(5)

In [ ]:
rdd_split = rdd.map(lambda line: line.split(' '))
rdd_split.take(5)

In [ ]:
features = rdd_split.map(lambda x: (float(x[0]), float(x[1]))).cache()  # Select only the features , convert to float
output = rdd_split.map(lambda x: int(x[2])).cache()  # Take the output

In [ ]:
features_python_list = features.collect()
# Convert it to a numpy array
features_numpy_array = array(features_python_list)
# Plot the figure using matplotlib library
plt.plot(features_numpy_array[:,0], features_numpy_array[:,1], 'b.')
plt.show()

In [ ]:
# Build the model (cluster the data)
cluster_model = KMeans.train(features, 8)  # intialization step is crucial in algorithms which are randomized
#cluster_model = KMeans.train(features, 8, initializationSteps=100, epsilon=1e-4)

In [ ]:
cluster_centers = array(cluster_model.clusterCenters)

plt.scatter(features_numpy_array[:,0], features_numpy_array[:,1]) # Plot the points first

for index in range(0,8):  # For every cluster plot the centers
    plt.scatter(cluster_centers[index,0], cluster_centers[index,1], c='r')
plt.show()

In [ ]:
# predict the output labels using the model generated
predicted_labels = array(cluster_model.predict(features).collect())
print(predicted_labels)  # have a look at the result

In [ ]:
# take out the unique output labels from array (Using numpy unique function)
numpy.unique(predicted_labels)

In [ ]:
# Now plot the scatter plot of the points again using the command above but with colors (Hint: use the parameter c to pass color array)
plt.scatter(features_numpy_array[:,0], features_numpy_array[:,1], c=predicted_labels)

In [ ]:
# Calculate the computational cost of the cluster
cluster_model.computeCost(features)